In [1]:
%matplotlib widget
import os, re
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.utils import make_grid
import torch.nn.functional as F
import copy

import lightly

from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import pandas as pd
from PIL import Image
import cv2
from sklearn.manifold import TSNE

import datasets
from datasets.wrgbd import WRGBD
import importlib

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
root_dir = '/home/ehajizad/ss_learning/neuromorphic-continual-learning'
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
##--------------------------------------------------------------------------------
## VGG-16, off-the-shelf, trained on ImageNet
vgg16 = torchvision.models.vgg16(weights='IMAGENET1K_V1')
# model = nn.Sequential(*list(vgg16.children())[:2], nn.Flatten(), *list(vgg16.children())[2][:2] )
feat_ext = vgg16.features

##--------------------------------------------------------------------------------
## ResNet-18, off-the-shelf, trained on ImageNet
feat_ext = torchvision.models.resnet18(weights='IMAGENET1K_V1')
feat_ext = nn.Sequential(*(list(feat_ext.children())[:-1]))

##--------------------------------------------------------------------------------
## Our on self-supervised trained models with ResNet-9 backbone
# resnet = lightly.models.ResNetGenerator('resnet-9')
# feat_ext = nn.Sequential(*list(resnet.children())[:-1],
#                          nn.AdaptiveAvgPool2d(2))

# feat_ext.load_state_dict(torch.load(
#         root_dir+"/models/"+model_name+".pth",
#         map_location=device))

feat_ext = feat_ext.to(device)
feat_ext.eval()
print("Model loeaded")

Model loeaded


In [4]:
from torchvision import models
from torchinfo import summary
# conv_feat_ext = feat_ext
# conv_feat_ext = nn.Sequential(*list(feat_ext.children())[:-2],nn.AdaptiveAvgPool2d(1))

# conv_feat_ext = nn.Sequential(*list(feat_ext.children())[:2],*list(feat_ext.children())[:-2][2][:-6])
#, *list(feat_ext.children())[2][:2]
# conv_feat_ext = nn.Sequential(*list(feat_ext.children())[:2], nn.Flatten(), *list(feat_ext.children())[2][:2] )
summary(feat_ext, (1,3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 512, 1, 1]            --
├─Conv2d: 1-1                            [1, 64, 64, 64]           9,408
├─BatchNorm2d: 1-2                       [1, 64, 64, 64]           128
├─ReLU: 1-3                              [1, 64, 64, 64]           --
├─MaxPool2d: 1-4                         [1, 64, 32, 32]           --
├─Sequential: 1-5                        [1, 64, 32, 32]           --
│    └─BasicBlock: 2-1                   [1, 64, 32, 32]           --
│    │    └─Conv2d: 3-1                  [1, 64, 32, 32]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 32, 32]           128
│    │    └─ReLU: 3-3                    [1, 64, 32, 32]           --
│    │    └─Conv2d: 3-4                  [1, 64, 32, 32]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 32, 32]           128
│    │    └─ReLU: 3-6                    [1, 64, 32, 32]           --
│

In [5]:
importlib.reload(datasets.wrgbd)
from datasets.wrgbd import WRGBD
from datasets.utils import SquarePad


# Dataset transform
transform=transforms.Compose([
    SquarePad(global_max_wh=160),
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

In [10]:
subsampling_factor = 5
depth_mask = False
obj_level = False
dataset_dir = '/home/ehajizad/datasets/rgbd-dataset'
n_run = 10
accs = np.zeros((10,1))
for i in range(n_run):
    seed = np.random.randint(1000)


    ## Extract training features
    train_ds = WRGBD(root_dir = dataset_dir, transform=transform, depth_mask = depth_mask, 
                     train_test_split='custom', subset='train',
                     obj_level=obj_level, subsampling_factor=subsampling_factor, seed=seed)
    embeddings = []
    labels = []

    feat_ext_dl = DataLoader(train_ds, batch_size=1, shuffle=False, num_workers=4)

    with torch.no_grad():
        for batch in feat_ext_dl:
            image, label = batch 
            image, label = image.to(device), label.to(device)
            emb = feat_ext(image).flatten(start_dim=1)
            embeddings.append(emb)
            labels.append(label)

    embeddings = torch.cat(embeddings, 0)
    embeddings = np.array(torch.Tensor.cpu(embeddings))

    labels = torch.cat(labels)
    labels = np.array(torch.Tensor.cpu(labels))

    ## Extract test features

    test_embeddings = []
    test_labels = []

    test_ds = WRGBD(root_dir = dataset_dir, transform=transform, depth_mask = depth_mask, 
                    train_test_split='custom', subset='test', 
                    obj_level=obj_level, subsampling_factor=subsampling_factor, seed=seed)

    feat_ext_dl = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=4)

    with torch.no_grad():
        for batch in feat_ext_dl:
            image, label = batch 
            image, label = image.to(device), label.to(device)
            emb = feat_ext(image).flatten(start_dim=1)
            test_embeddings.append(emb)
            test_labels.append(label)

    test_embeddings = torch.cat(test_embeddings, 0)
    test_embeddings = np.array(torch.Tensor.cpu(test_embeddings))

    test_labels = torch.cat(test_labels)
    test_labels = np.array(torch.Tensor.cpu(test_labels))

    X_train, X_test = embeddings.copy(), test_embeddings.copy()
    y_train, y_test = labels.copy(), test_labels.copy()

    ###----------------------------------------------------------###
    ###-----Offline Object Identifiation Exepriment--------------###


    clf = svm.LinearSVC(max_iter=5000)
    clf.fit(X_train, y_train)

    # y_pred_train=clf.predict(X_train)
    y_pred_test=clf.predict(X_test)

    # Model Accuracy: how often is the classifier correct?
    # print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
    acc = metrics.accuracy_score(y_test, y_pred_test)
    accs[i] = acc
    print("Test Accuracy:", acc)

All target lists are created
All target lists are created


/home/ehajizad/ss_learning/ssl_env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test Accuracy: 0.785622040464916
All target lists are created
All target lists are created


/home/ehajizad/ss_learning/ssl_env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test Accuracy: 0.7771535580524345
All target lists are created
All target lists are created


...

/home/ehajizad/ss_learning/ssl_env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test Accuracy: 0.7968524400808548
All target lists are created
All target lists are created


/home/ehajizad/ss_learning/ssl_env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test Accuracy: 0.8145667046750285
All target lists are created
All target lists are created
Test Accuracy: 0.7800143781452192


/home/ehajizad/ss_learning/ssl_env/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [12]:
mean_acc = np.round(100*np.mean(accs),1)
std_acc = np.round(100*np.std(accs),1)
print("Offline Object Categorization Exepriment accuracies:", str(mean_acc), u"\u00B1", str(std_acc))


Offline Object Categorization Exepriment accuracies: 79.3 ± 1.5
